# This is an example of doing the geocoding we did with OpenRefine but using Python to call the same service

In [ ]:
#add your app_id here
APP_ID = '48eab85b'
#add your app_key here
APP_KEY = 'fa31219b4e426ff96a8ded2905e6ae3a'

###We have to tell Python where to find the input file, giving the full path to the file as the first argument and then whether it's reading the file ('r') or writing to the file ('w'). We could also append to a file ('a')
###For more information, see http://www.tutorialspoint.com/python/python_files_io.htm

In [ ]:
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') #change to match the file location
header = next(inputFile) #take the next line of input and assign it to a variable

In [ ]:
header #the value of the variable header

In [ ]:
import csv #package for dealing with csv files

###We use the csv.reader function to read in the columns properly, especially the ones that have commas in between double quotation marks.  This is a built in package to Python, but we still have to link to it with the import statement above
###For more information, see https://docs.python.org/2/library/csv.html

In [ ]:
inputReader = csv.reader(inputFile) 

In [ ]:
#10 lines from the file
for i in range(10):
    print next(inputReader)

In [ ]:
test_line = next(inputReader) #the next function returns a line and I'm assigning that line to a variable. 
# This helps me test the input from the text.
test_line

In [ ]:
test_line[1]

In [ ]:
#because we've already iterated through some of the file contents, it's best to re-open the file and start from the beginning
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [ ]:
#this time, we're just printing out the street address
for line in inputReader:
    print line[1]
inputFile.close() #close the open input file

In [ ]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

###This time, we're taking the address and inserting it into the API string I can check the string by clicking the link and it should provide me the response I'm looking for this allows me to test that I've constructed the string properly

In [ ]:
for line in inputReader:
    houseNumber = line[1].split(" ")[0] #take the first segment of the address
    street = ' '.join(line[1].split(" ")[1:]) #take the rest of the address as the street
    zipcode = line[4][:5]
    print('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)

inputFile.close() #close the open input file

##Requests package for executing web requests [('HTTP for Humans')](http://docs.python-requests.org/en/latest/)

In [ ]:
import requests 

In [ ]:
#This is a test call using the values from the last loop using requests.get
r = requests.get('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)

###Just like with OpenRefine, we have to navigate the tree. The 'u' before the string indicates it's in [Unicode format](http://en.wikipedia.org/wiki/Unicode_symbols)
###I use this test response to know which fields to use in my for loop later on

In [ ]:
r.json()

In [ ]:
r.json()['address']

In [ ]:
r.json()['address']['latitude'], r.json()['address']['longitude']

In [ ]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [ ]:
#This time, I want to process the response I get. I'm going to get the response and write the output to a file
#Here I'm opening the file I'm going to write to. If the file already exists, it overwrite the file.
outputFile = open('Lincoln_Square_BID_Business_List_geocoded.csv','w') #opens the file
outputFile.write(header.rstrip() + ',longitude,latitude\n') #writes the first line of my CSV file. NOTE: '\n' writes a new line

###This time through we put the pieces together.  I'm executing the web request, I'm getting the reponse, and then writing it to the output

In [ ]:
for line in inputReader:
    houseNumber = line[1].split(" ")[0] #take the first segment of the address
    street = ' '.join(line[1].split(" ")[1:]) #take the rest of the address as the street
    zipcode = line[4][:5]
    #the web request, using the get() function from the package requests and assigning it to the variable r
    r = requests.get('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)
    
    
    #My default value is 'Null' for longitude. 0 is a legitimate value and when I map these, I don't want anything to plot to 0,0
    longitude = 'Null' 
    latitude = 'Null'

    
    #longitude
    if 'longitude' in r.json()['address'].keys(): #I first check that there is a longitude present
        #if it's present, I assign the value to the variable "longitude". Otherwise, the variable longitude has the value 'Null'
        longitude = r.json()['address']['longitude'] 
    #latitude
    if 'latitude' in r.json()['address'].keys(): #I then check that there is a latitude present
        #if it's present, I assign the value to the variable "latitude". Otherwise, the variable latitude has the value 'Null'
        latitude = r.json()['address']['latitude']
    #I then write the result to the output file, stringing together the variables
    #because the latitude and longitude are decimal numbers, I have to convert them to strings using the str() function
    #in order to combine them together with the rest of the variables. 
    #Note the commas in the string and the '\n' new line character at the end of the string
    outputFile.write(','.join(line) + ',' + str(longitude) + ',' + str(latitude) + '\n')

inputFile.close() #close the input file
outputFile.close() #close the open output file. 
#This is important because if you don't close it, it's possible the last bit of data won't get written to the file before the 
#program closes.

##And we're done...

<img src="http://www.quickmeme.com/img/a5/a5e4a5a47029bf5fb99de6baa5ac7c9a664c3cd0b9c40d67d49c4c591dde0282.jpg">